Using data from https://www.kaggle.com/datasets/kinguistics/heartbeat-sounds

with [librosa](https://librosa.org/doc/latest/index.html) audio processing library.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load metadata table for heartbeat audio files.

In [46]:
meta = pd.read_csv('set_b.csv')
meta.head()

,dataset,fname,label,sublabel
0,b,set_b/Btraining_extrastole_127_1306764300147_C...,extrastole,NaN
1,b,set_b/Btraining_extrastole_128_1306344005749_A...,extrastole,NaN
2,b,set_b/Btraining_extrastole_130_1306347376079_D...,extrastole,NaN
3,b,set_b/Btraining_extrastole_134_1306428161797_C...,extrastole,NaN
4,b,set_b/Btraining_extrastole_138_1306762146980_B...,extrastole,NaN


Load audio files and store as a list of dataframes.

In [53]:
import librosa as lr

rate = 1000  # resampling rate / Hz

data = []

for index, row in meta.iterrows():
    fname = row['fname']#.values[0]
    fname = fname.replace('Btraining_','')
    fname = fname.replace('extrastole_','extrastole__')
    fname = fname.replace('murmur_','murmur__')
    fname = fname.replace('normal_','normal__')
    if( not ( fname.startswith('set_b/Bunlabelled') or \
              fname.startswith('set_b/murmur__noisy') or \
              fname.startswith('set_b/normal__noisy'))) :
        audio, sfreq = lr.load( fname, sr=rate )
        s = pd.Series({'label':row['label'],    # classification label
                       'audio':audio,          # audio data as float array
                      'length':len(audio)})   # length in ms
        data.append(s.to_frame().T)
 

In [59]:
len(data)

312

Convert to a single dataframe.

In [62]:
df = pd.concat(data).reset_index(drop=True)
df.head()

,label,audio,length
0,extrastole,"[0.016320875, 0.02191445, 0.015383912, 0.01628...",4676
1,extrastole,"[0.0115379, 0.0042967927, 0.006828971, 0.01598...",9237
2,extrastole,"[-0.012741837, -0.015750436, -0.0045903595, 5....",4003
3,extrastole,"[0.0025598854, 0.0032684598, 3.7724152e-05, -0...",2562
4,extrastole,"[0.012491666, 0.0048830546, -0.0061410083, 0.0...",8104


Save as JSON.

In [63]:
df.to_json('heartbeat.json')